In [ ]:
import subprocess

partitioning = ("small_balanced", "small_random", "mid_balanced", "mid_random", "large_balanced", "large_random")
algorithms = ("fedavg", "fedprox")
communication_rounds = (20, 30)

outputs = {p: {a: {c: None for c in communication_rounds} for a in algorithms} for p in partitioning}

for partition in partitioning:
    for alg in algorithms:
        for rounds in communication_rounds:
            command = f"python experiments.py --dataset=clinical --model=mlp --device=cpu --comm_round={rounds} --n_parties=10 --partition={partition} --alg={alg}"
            print(f"Executing `{command}`")
            process = subprocess.Popen(command.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            _, output = process.communicate()
            outputs[partition][alg][rounds] = output

In [ ]:
import re


def extract_test_history(outputs_: dict[str, dict[int, dict[str, bytes]]]) -> dict[
    str, dict[int, dict[str, list[float]]]]:
    result = {p: {a: {c: [] for c in communication_rounds} for a in algorithms} for p in partitioning}
    regex = "net \d+ final test acc (\d+[.]\d+)"
    for partition in partitioning:
        for alg in algorithms:
            for rounds in communication_rounds:
                snippet = outputs_[partition][alg][rounds].decode().split("in comm round:")
                for subsnippet in snippet[1:]:
                    result[partition][alg][rounds].append(
                        re.findall(
                            regex,
                            subsnippet
                        )
                    )
    return result

In [ ]:
extract_test_history(outputs)